# 要約 
このJupyter Notebookは、「20の質問」ゲームにおける言語モデル（LLM）のエージェントの構築に関するもので、特に質問者と回答者の二つのエージェントに焦点を当てています。具体的には、エージェントが与えられたキーワードに基づいて質問を行ったり、回答を返したりするロジックを実装しています。

## 取り組む問題
- **問題**: 「20の質問」ゲームにおいて、質問者エージェントと回答者エージェントが効果的に相互作用し、限られた質問数で正解のキーワードを推測する能力を向上させることに取り組んでいます。ゲームの制約に対応し、適切な質問と推測を行う必要があります。

## 使用している手法・ライブラリ
1. **ライブラリ**:
   - `pandas`: データフレームを使用してキーワード情報を整理するために利用。
   - `json`: JSON形式のデータを読み込むために使用。
   - `torch`: 深層学習モデルのためのPyTorchライブラリ。
   - `immutabledict`と`sentencepiece`: モデルの依存関係としてインポート。

2. **クラスと構造**:
   - **GemmaFormatter**: ユーザープロンプトやモデルレスポンスのフォーマットを管理し、現在の会話状態を保持。
   - **Observation**: 課題に対する観察結果を管理し、過去の質問・回答・推測をトラック。
   - **GemmaAgent**: 基底クラスとして、質問者と回答者エージェントの基本的な機能を提供。
   - **GemmaQuestionerAgent**: 質問者エージェントとしての具体的なロジックを実装。
   - **GemmaAnswererAgent**: 回答者エージェントとしてのロジックを実装。

3. **システムプロンプトと少数ショット例**:
   - システムプロンプトと少数ショット例は、エージェントの動作を導くために設定され、ゲームの目的やエージェントの役割について明示されています。

4. **エージェントの実装**:
   - エージェントは動的に初期化され、観察情報に基づいて適切な動作（質問する、回答する、推測する）を行います。

このように、Notebookは「20の質問」ゲームを支援するためのLLMエージェントを構築するための多くの要素を組み合わせており、自然言語処理によるインタラクションを促進するために設計されています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連する専門用語の簡単な解説です。特に、初心者がつまずきやすいマイナーな用語やドメイン特有の知識に焦点を当てています。

1. **Gemma**:
   - Gemmaは、特定の自然言語処理（NLP）タスク、特に因果推論（Causal Language Modeling）に焦点を当てた言語モデルです。このNotebookで使われるGemmaは、モデルの振る舞いを特定の目的に合わせて訓練・調整するために使用されています。

2. **Causal LM (因果言語モデル)**:
   - 過去の単語に基づいて次の単語を予測するタイプの言語モデルです。一方向に時間を考慮し、コンテキストの保持が重要です。

3. **contextlib**:
   - Pythonの標準ライブラリで、特定のコードブロックの前後で環境を管理するためのユーティリティを提供します。例えば、ファイルのオープンとクローズ、または特定の設定を一時的に変更するのに使われます。

4. **Pydantic**:
   - Pythonのデータバリデーションと設定管理を簡素化するためのライブラリです。データモデルを定義する際に、型ヒントを使って簡単にデータの検証や変換を行うことができます。

5. **torch.set_default_dtype()**:
   - PyTorchにおいて、全てのテンソルのデフォルトのデータ型を指定するためのメソッドです。デフォルトのデータ型を変えることで、メモリ使用量や計算の精度を調整することができます。

6. **few-shot examples (数ショット例)**:
   - モデルを訓練する際に、数少ないサンプルから学習するためのアプローチです。特に、NLPタスクにおいては、数例の入力と出力の組み合わせを用いて、モデルの応答を制御するのに役立ちます。

7. **Observations (観察)**:
   - ゲームの状態やエージェントのアクションを表すためのデータ構造です。ゲームの進行に応じた質問、回答、推測などが含まれ、エージェントがそれらの情報を用いて次のアクションを決定します。

8. **interleave_unequal**:
   - 二つのリストを交互に結合し、片方のリストが短い場合は対応する要素がない部分は無視する関数です。この動作により、異なる長さのリストを扱う際に便利です。

9. **@property decorator**:
   - クラス内のメソッドをプロパティとして扱うためのデコレータです。この機能を使うことで、クラスの属性に対してカスタムのゲッターを提供し、属性にアクセスする際に特定のロジックを組み込むことができます。

10. **set_default_tensor_type**:
    - PyTorchにおいて、全ての新しいテンソルに適用されるデフォルトのデータタイプを設定するためのコンテクストマネージャです。この機能を用いることで、特定のデータ型で処理を行なうことが容易になります。

これらの用語は、特に初心者や実務経験のない人にとっての理解を助けるためのものです。各用語の背景や実際の利用方法について掘り下げて学ぶことをお勧めします。

---


In [ ]:
# pandasライブラリをpdという名前でインポートします
# jsonライブラリをインポートします

# キーワードをロードします
from importlib.machinery import SourceFileLoader
# keywords.pyファイルからキーワードを読み込みます
keywords = SourceFileLoader("keywords",'/kaggle/input/llm-20-questions/llm_20_questions/keywords.py').load_module()
# JSON形式のキーワードデータを読み込みます
df = json.loads(keywords.KEYWORDS_JSON)

words = []
# カテゴリごとにキーワードをループ処理します
for cat in df:
    # カテゴリ名とその単語数を表示します
    print(cat['category'], len(cat['words']))
    # 各キーワードをリストに追加します
    for w in cat['words']:
        words.append([cat['category'], w["keyword"], w["alts"], "", "", 0.0, 0.0])

# キーワードをデータフレームに変換します
df = pd.DataFrame(words, columns=['Category','Word','Alternatives', "Cat1", "Cat2", "Lat", "Lon"])
# データフレームの最初の5行を表示します
df.head()

In [ ]:
%%bash
# 作業ディレクトリに移動します
cd /kaggle/working
# 作業ディレクトリ内のファイルをすべて削除します
rm -rf ./*
# immutabledictとsentencepieceを指定のディレクトリにインストールします
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
# gemma_pytorchリポジトリをクローンします
git clone https://github.com/google/gemma_pytorch.git > /dev/null
# gemmaのディレクトリを作成します
mkdir /kaggle/working/submission/lib/gemma/
# gemma_pytorch内のファイルを新しいディレクトリに移動します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%writefile submission/main.py
# # セットアップ
import os
import sys

# Kaggleエージェントのパスを設定します
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    # Kaggle環境のライブラリパスをシステムパスに追加します
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    # ワーキングディレクトリのライブラリパスをシステムパスに追加します
    sys.path.insert(0, "/kaggle/working/submission/lib")

import contextlib
import os
import sys
from pathlib import Path

import torch
from gemma.config import get_config_for_7b, get_config_for_2b
from gemma.model import GemmaForCausalLM

if os.path.exists(KAGGLE_AGENT_PATH):
    # モデルの重みパスを設定します
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/7b-it-quant/2")
else:
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/7b-it-quant/2"


import itertools
from typing import Iterable


class GemmaFormatter:
    _start_token = '<start_of_turn>'
    _end_token = '<end_of_turn>'

    # 初期化
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self._turn_user = f"{self._start_token}user\n{{}}{self._end_token}\n"
        self._turn_model = f"{self._start_token}model\n{{}}{self._end_token}\n"
        self.reset()

    # 現在の会話状態を文字列として返します
    def __repr__(self):
        return self._state

    # ユーザープロンプトを会話に追加します
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
        
    # モデルのレスポンスを会話に追加します
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self

    # ユーザのターンの開始を記録します
    def start_user_turn(self):
        self._state += f"{self._start_token}user\n"
        return self

    # モデルのターンの開始を記録します
    def start_model_turn(self):
        self._state += f"{self._start_token}model\n"
        return self

    # 現在のターンの終了を記録します
    def end_turn(self):
        self._state += f"{self._end_token}\n"
        return self

    # リセットメソッド
    def reset(self):
        # `_state`を空の文字列に初期化します
        self._state = ""  

        # 提供された場合はシステムプロンプトを追加します
        if self._system_prompt is not None:
            self.user(self._system_prompt)  
            
        # 提供された場合はfew-shot例を適用します
        if self._few_shot_examples is not None: 
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        # 発言を使うエージェントの順番を決めます
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self


import re

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """デフォルトのtorchデータ型を指定されたdtypeに設定します。"""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

from pydantic import BaseModel
    
class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        # 質問、回答、推測がすべて空であるかをチェックします
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        # 過去の質問、回答、推測をまとめて返します
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            # 過去のやり取りの履歴を文字列として整形します
            history = "\n".join(
            f"""{'**Question:** ' + question if not skip_question else ''} -> {'**Answer:** ' + answer if not skip_answer else ''}
            {'**Previous Guess:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "まだありません。"


class GemmaAgent:
    def __init__(self, variant='7b-it-quant', device='cuda:0', system_prompt=None, few_shot_examples=None):
        self._variant = variant
        self._device = torch.device(device)
        self.formatter = GemmaFormatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)

        print("モデルを初期化しています")
        
        # モデル設定
        model_config = get_config_for_2b() if "2b" in variant else get_config_for_7b()
        model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")
        model_config.quant = "quant" in variant
        
        # モデルの初期化
        with _set_default_tensor_type(model_config.get_dtype()):
            """TODO: モデルを変更してパフォーマンスを見てみる"""
            model = GemmaForCausalLM(model_config)
            ckpt_path = os.path.join(WEIGHTS_PATH , f'gemma-{variant}.ckpt')
            model.load_weights(ckpt_path)
            self.model = model.to(self._device).eval()

    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(f"{response=}")
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError

    def _call_llm(self, prompt, max_new_tokens=32, **sampler_kwargs):
        """TODO: LLMのパラメータを変更します"""
        if sampler_kwargs is None:
            sampler_kwargs = {
                'temperature': 0.01,
                'top_p': 0.1,
                'top_k': 1,
        }
        response = self.model.generate(
            prompt,
            device=self._device,
            output_len=max_new_tokens,
            **sampler_kwargs,
        )
        return response

    def _parse_keyword(self, response: str):
        # レスポンスからキーワードを抽出します
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError


def interleave_unequal(x, y):
    # 2つのリストを交互に結合し、不足している要素を除外します
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]

"""TODO: 質問者を調整します"""
class GemmaQuestionerAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        # セッションを開始します
        self.formatter.reset()
        self.formatter.user("20の質問をプレイします。あなたは質問者としての役割を担います。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        hints = self.turns_to_hints(turns)
        if obs.turnType == 'ask':
            if len(turns) == 0:
                self.formatter.user("はいかいいえで答えられる質問をしてください。")
            else:
                self.formatter.user(f"このリストのキーワードのヒントに基づいて、はいかいいえで答えられる質問をしてください: {hints}")
        elif obs.turnType == 'guess':
            self.formatter.user("今、キーワードを推測してください。あなたの推測を二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            # ユーザーの質問を抽出します
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None:
                question = "人ですか？"
            else:
                question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            return guess
        else:
            raise ValueError("不明なターンタイプ:", obs.turnType)
    
    # 質問を肯定または否定のステートメントに変換するためにターンを変換します
    def turns_to_hints(self, turns):
        hints = []
        for i in range(0, len(turns), 2):
            question = turns[i]
            answer = turns[i+1]
            # hints.append(f"Question: {question} Answer: {answer}")
            
            if question.startswith('is it'):
                item = question[6:-1]  # "is it "の後と"?"の前の部分を抽出します
                if answer == 'yes':
                    transformed_statement = f"それは{item}です"
                else:
                    transformed_statement = f"それは{item}ではありません"
            elif question.startswith('does'):
                item = question[5:-1]  # "does "の後と"?"の前の部分を抽出します
                if answer == 'yes':
                    transformed_statement = f"それは{item}をします"
                else:
                    transformed_statement = f"それは{item}をしません"
            else:
                transformed_statement = f"認識できない質問形式: {question}"

            hints.append(transformed_statement)

        return hints


class GemmaAnswererAgent(GemmaAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _start_session(self, obs):
        self.formatter.reset()
        # キーワードとカテゴリーに基づいて初期ユーザー発言をフォーマットします
        self.formatter.user(f"20の質問をプレイします。あなたは回答者の役割を担います。キーワードは{obs.keyword}で、カテゴリは{obs.category}です。")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"このキーワード{obs.keyword}とそのカテゴリ{obs.category}に関する質問です。はいまたはいいえで答え、あなたの答えを二重アスタリスクで囲んでください。")
        self.formatter.start_model_turn()

    def _parse_response(self, response: str, obs: dict):
        # レスポンスから回答を取得します
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

# エージェントの作成
system_prompt = """あなたは20の質問ゲームをプレイするためのAIアシスタントです。 
このゲームでは、回答者がキーワードを考え、質問者がはいかいいえで質問します。キーワードは特定の人、場所、または物です。あなたの目的は、できるだけ少ないターンで秘密のキーワードを推測し、勝つことです。"""

few_shot_examples = [
    "20の質問をプレイします。あなたは質問者の役割を担います。最初の質問をしてください。",
    "人ですか？", "**いいえ**",
    "場所ですか？", "**はい**",
    "国ですか？", "**はい** 今、キーワードを推測してください。",
    "**フランス**", "正解!",
]

# **重要:** エージェントをグローバルに定義して、必要なエージェントのみをロードします。
# 両方をロードすると、メモリ不足になる可能性があります。

# エージェント変数を初期化します
agent = None

# 要求されたエージェントに基づいて適切なエージェントを取得するための関数
def get_agent(name: str):
    global agent
    
    # エージェントが初期化されておらず、要求されたエージェントが「質問者」の場合
    if agent is None and name == 'questioner':
        # GemmaQuestionerAgentを特定のパラメータで初期化します
        agent = GemmaQuestionerAgent(
            device='cuda:0',  # 計算用のデバイス
            system_prompt=system_prompt,  # エージェントのシステムプロンプト
            few_shot_examples=few_shot_examples,  # エージェントの振る舞いを導く例
        )
    # エージェントが初期化されておらず、要求されたエージェントが「回答者」の場合
    elif agent is None and name == 'answerer':
        # GemmaAnswererAgentを同じパラメータで初期化します
        agent = GemmaAnswererAgent(
            device='cuda:0',
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    
    # エージェントが初期化されていることを確認します
    assert agent is not None, "エージェントが初期化されていません。"

    # 初期化されたエージェントを返します
    return agent

# 観察に基づいてインタラクションを処理するための関数
def agent_fn(obs, cfg):
    # 観察が質問するためのものである場合
    if obs.turnType == "ask":
        # 「質問者」エージェントを使用して観察に応答します
        response = get_agent('questioner')(obs)
    # 観察が推測するためのものである場合
    elif obs.turnType == "guess":
        # 「質問者」エージェントを使用して観察に応答します
        response = get_agent('questioner')(obs)
    # 観察が回答を提供するためのものである場合
    elif obs.turnType == "answer":
        # 「回答者」エージェントを使用して観察に応答します
        response = get_agent('answerer')(obs)
    
    # エージェントからのレスポンスがNoneまたは非常に短い場合
    if response is None or len(response) <= 1:
        # ポジティブなレスポンス（「はい」）を仮定します
        return "はい"
    else:
        # エージェントから受け取ったレスポンスを返します
        return response

In [ ]:
# pigzとpvをインストールします
!apt install pigz pv > /dev/null
# # 提出物をtar.gz形式で圧縮します（圧縮プログラムとしてpigzを使用します）
# !tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/gemma/pytorch/7b-it-quant/2

# 提出物をtar.gz形式で圧縮します（圧縮プログラムとしてpigzを使用します）
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/7b-it-quant/2

In [ ]:
# obsというクラスを定義します
#     '''
#     このクラスは、ゲームからの観察を表します。 
#     turnTypeは['ask','answer','guess']のいずれかでなければなりません。
#     テストしたいキーワードを選択します。
#     キーワードのカテゴリを選択します。
#     質問は文字列のリストです。エージェントに質問履歴を渡す場合。
#     回答は文字列のリストです。
#     responseを使用して、質問を回答エージェントに渡します。
#     '''
#     def __init__(self, turnType, keyword, category, questions, answers):
#         self.turnType = turnType
#         self.keyword = keyword
#         self.category = category
#         self.questions = questions
#         self.answers = answers

# #エージェントに渡すためのobsの例。自分のobsをここに渡します。
# obs1 = obs(turnType='ask', 
#           keyword='paris', 
#           category='place', 
#           questions=['人ですか？', '場所ですか？', '国ですか？', '都市ですか？'], 
#           answers=['いいえ', 'はい', 'いいえ', 'はい'])

# question = agent_fn(obs1, None)
# print(question)

# #エージェントからのレスポンスが出力されるはずです。 
# #提出エラーの解決には役立ちませんが、エージェントのレスポンスを評価するのに役立ちます。